<a href="https://colab.research.google.com/github/ErinMao233/Network-model-output-analysis/blob/main/Network_Output_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**What's this for**

1.   This is to analyze demand model output, and translate it to user friendly report
2.   According to distance, group ICs to 9 groups
3.   Compare this time model's period 5 with last time model's period 4 data for # of trips and hauler miles metrics
4.   List details of Origin and destination pairs








In [ ]:
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
import google.auth
credentials = google.auth.default()
from gspread_dataframe import get_as_dataframe, set_with_dataframe

auth.authenticate_user()
gauth = GoogleAuth()
drive = GoogleDrive(gauth)


from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_df_from_worksheet(spreadsheet, worksheet_name):
    worksheet = spreadsheet.worksheet(worksheet_name)
    rows = worksheet.get_all_values()
    headers = rows.pop(0)
    df = pd.DataFrame(rows, columns=headers)
    return df

In [ ]:
# read all data from spreadsheet
# read Last PC utilization_details_weekly_adj
spreadsheet_key_last_uti='11r1OX29yC5YZ0UPeiqFrGNdpdOMmYEunzF0c1s-hdUs'
spreadsheet_last_uti = gc.open_by_key(spreadsheet_key_last_uti)
df_last_PC_uti = get_df_from_worksheet(spreadsheet_last_uti, 'utilization_details_weekly')

# read Current PC utilization_details_weekly_adj
spreadsheet_key_curr_uti='1eZh1Iu3rHxVly2fLCtsUJzAFuimod8ocX6JSLp_ZBM0'
spreadsheet_curr_uti = gc.open_by_key(spreadsheet_key_curr_uti)
df_curr_PC_uti = get_df_from_worksheet(spreadsheet_curr_uti, 'utilization_details_weekly')

# # read IC grouping spreadsheet
# spreadsheet_key_grouping='1DuSvbQEtpRHY_hMKx7sSJOOwweEJSZluOL3cqBhrOxg'
# spreadsheet_grouping = gc.open_by_key(spreadsheet_key_grouping)
# df_IC_grouping = get_df_from_worksheet(spreadsheet_grouping, 'IC Group')

# # read Current trip_limits
# spreadsheet_key_curr_uti='1_cT6Xp-6_6hKncCDkoafiMCP8CVl2eqOaiD5k2V6RDo'
# spreadsheet_curr_uti = gc.open_by_key(spreadsheet_key_curr_uti)
# df_curr_trip_limits = get_df_from_worksheet(spreadsheet_curr_uti, 'trip_limits')

# # read last trip_limits
# spreadsheet_key_last_uti='1szt1hFJKL59eDfvIRXLcB_OWOXoXZpO9UQv-d6KcXrI'
# spreadsheet_last_uti = gc.open_by_key(spreadsheet_key_last_uti)
# df_last_trip_limits = get_df_from_worksheet(spreadsheet_last_uti, 'trip_limits')

In [ ]:
df_IC_grouping = pd.DataFrame({
    'Group': ['Group1', 'Group1', 'Group1', 'Group1', 'Group1', 'Group1',
              'Group2',
              'Group3', 'Group3', 'Group3', 'Group3',
              'Group4', 'Group4', 'Group4', 'Group4',
              'Group5', 'Group5', 'Group5', 'Group5', 'Group5',
              'Group6', 'Group6',
              'Group7', 'Group7', 'Group7',
              'Group8',
              'Group9', 'Group9', 'Group9', 'Group9', 'Group9', 'Group9', 'Group9'],
    'IC': ['ARVMH', 'BANLH', 'LNVMH', 'ROCIC', 'TOL', 'TOOIC',
           'POSMH',
           'BM', 'HTNLH', 'OKCIC', 'SAAIC',
           'ABUMH', 'BMAIC', 'DEL', 'NNJIC',
           'ELYIC', 'HEAIC', 'INDIC', 'TREIC', 'UNIIC',
           'CHEIC', 'CONIC',
           'BESIC', 'WIND', 'WMEIC',
           'HNSIC',
           'BIDLH', 'DEN', 'EUGLH', 'KSC', 'MINN', 'MOBLH', 'MOR']
})

In [ ]:
# Convert all 2 tables column to numeric
df_last_PC_uti[['period', 'number_of_cars_weekly', 'round_trip_distance', 'hauler_miles']] = \
df_last_PC_uti[['period', 'number_of_cars_weekly', 'round_trip_distance', 'hauler_miles']].apply(pd.to_numeric)
df_curr_PC_uti[['period', 'number_of_cars_weekly', 'round_trip_distance', 'hauler_miles']] = \
df_curr_PC_uti[['period', 'number_of_cars_weekly', 'round_trip_distance', 'hauler_miles']].apply(pd.to_numeric)

In [ ]:
# design a function to have loop, origins, period, round_trip_distance, hauler_miles, number_of_trips, Group 7 columns
def prepare_curr_df(df_uti_name):
 df_uti = df_uti_name[(df_uti_name['period'] == 5) & (df_uti_name['round_trip_distance'] != 0)]
 df_uti = df_uti[['loop', 'origins', 'period', 'round_trip_distance', 'hauler_miles', 'number_of_cars_weekly']]
 df_uti = df_uti.merge(df_IC_grouping, how = 'left', left_on = 'origins', right_on = 'IC')
 df_uti['number_of_trips'] = np.ceil((df_uti['number_of_cars_weekly']/9).round(1))
 df_uti.drop(['IC', 'number_of_cars_weekly'], axis = 1, inplace = True)
 return df_uti

def prepare_last_df(df_uti_name):
 df_uti = df_uti_name[(df_uti_name['period'] == 4) & (df_uti_name['round_trip_distance'] != 0)]
 df_uti = df_uti[['loop', 'origins', 'period', 'round_trip_distance', 'hauler_miles', 'number_of_cars_weekly']]
 df_uti = df_uti.merge(df_IC_grouping, how = 'left', left_on = 'origins', right_on = 'IC')
 df_uti['number_of_trips'] = np.ceil((df_uti['number_of_cars_weekly']/9).round(1))
 df_uti.drop(['IC', 'number_of_cars_weekly'], axis = 1, inplace = True)
 return df_uti

# get clean data for 2 raw tables
df_curr_PC_uti = prepare_curr_df(df_curr_PC_uti)
df_last_PC_uti = prepare_last_df(df_last_PC_uti)
# df_curr_PC_uti

In [ ]:
# make destination column
df_last_PC_uti['destination'] = df_last_PC_uti['loop'].str.split(',').apply(lambda x: x[1])
df_curr_PC_uti['destination'] = df_curr_PC_uti['loop'].str.split(',').apply(lambda x: x[1])

In [ ]:
#Create Is_destination_An_Origin column
def Create_IsOrigin_col(df_uti_name):
  df_uti = df_uti_name
  found = False
  Is_destination_An_Origin = []
  for j in range(0, len(df_uti)):
    for i in range(0, len(df_IC_grouping)):
        if df_uti['destination'].iloc[j] == df_IC_grouping['IC'].iloc[i]:
          found = True
          Is_destination_An_Origin.append(True)
          break
    else:
      Is_destination_An_Origin.append(False)
  df_uti['Is_destination_An_Origin'] = pd.Series(Is_destination_An_Origin)



Create_IsOrigin_col(df_last_PC_uti)
Create_IsOrigin_col(df_curr_PC_uti)
# df_curr_PC_uti

In [ ]:
def Create_ODPair(df_uti_name):
  df_uti = df_uti_name
  OD_pair = []
  for i in range(0, len(df_uti)):
    if df_uti['Is_destination_An_Origin'].iloc[i] == False:
      OD_pair.append(df_uti['origins'].iloc[i] + ',' + df_uti['destination'].iloc[i])
    elif df_uti['origins'].iloc[i] <= df_uti['destination'].iloc[i]:
      OD_pair.append(df_uti['origins'].iloc[i] + ',' + df_uti['destination'].iloc[i])
    else:
      OD_pair.append(df_uti['destination'].iloc[i] + ',' + df_uti['origins'].iloc[i])
  df_uti['OD_pair'] = pd.Series(OD_pair)
Create_ODPair(df_last_PC_uti)
Create_ODPair(df_curr_PC_uti)

In [ ]:
df_PC_comp = df_curr_PC_uti.merge(df_last_PC_uti, left_on = ['origins', 'period', 'OD_pair', 'round_trip_distance', 'Group', 'loop']\
                    , right_on = ['origins', 'period', 'OD_pair', 'round_trip_distance', 'Group', 'loop'], how = 'outer')
# df_PC_comp = df_curr_PC_uti.merge(df_last_PC_uti, left_on = ['OD_pair']\
#                     , right_on = ['OD_pair'], how = 'left')
df_PC_comp.rename(columns = {"number_of_trips_x": "number_of_trips_curr_PC", "number_of_trips_y": "number_of_trips_last_PC",\
                             "hauler_miles_x": "hauler_miles_curr_PC", "hauler_miles_y": "hauler_miles_last_PC",\
                             "destination_x": "destination_curr_PC", "destination_y": "destination_last_PC"}, inplace = True)
df_PC_comp.drop(columns = ['Is_destination_An_Origin_x', 'Is_destination_An_Origin_y'], inplace = True)
df_PC_comp['number_of_trips_curr_PC'].fillna(0, inplace = True)
df_PC_comp['number_of_trips_last_PC'].fillna(0, inplace = True)
df_PC_comp['hauler_miles_curr_PC'].fillna(0, inplace = True)
df_PC_comp['hauler_miles_last_PC'].fillna(0, inplace = True)


In [ ]:
path = '/content/drive/My Drive/output.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_PC_comp.to_csv(f)

In [ ]:
# # get last PC origin level trips df
# df_last_PC_uti_4 = df_last_PC_uti[(df_last_PC_uti['period'] == 4) & (df_last_PC_uti['round_trip_distance'] != 0)]
# df_last_PC_uti_4 = df_last_PC_uti_4.groupby('origins').sum()['number_of_cars_weekly']/9

# df_curr_PC_uti_4 = df_curr_PC_uti[(df_curr_PC_uti['period'] == 4) & (df_curr_PC_uti['round_trip_distance'] != 0)]
# df_curr_PC_uti_4 = df_curr_PC_uti_4.groupby('origins').sum()['number_of_cars_weekly']/9

# df_PC_comp = df_curr_PC_uti_4.to_frame().merge(df_last_PC_uti_4, on = 'origins', how = 'left')
# df_PC_comp.rename(columns = {"number_of_cars_weekly_x": "Num_trips_curr_PC", "number_of_cars_weekly_y": "Num_trips_last_PC"}, inplace = 'True')
# df_PC_comp['Diff'] = df_PC_comp['Num_trips_curr_PC'] - df_PC_comp['Num_trips_last_PC']
# df_PC_comp['Diff %'] = (df_PC_comp['Num_trips_curr_PC'] - df_PC_comp['Num_trips_last_PC']) / df_PC_comp['Num_trips_last_PC']

